In [9]:
import sys
print(sys.executable)
!pip list
import xbrl

/Users/samuelzobev/micromamba/envs/.venv/bin/python
Package                      Version
---------------------------- ----------
absl-py                      2.0.0
aiohttp                      3.8.4
aiosignal                    1.3.1
alpha-vantage                2.3.1
appdirs                      1.4.4
appnope                      0.1.3
asttokens                    2.4.1
astunparse                   1.6.3
async-timeout                4.0.2
attrs                        22.2.0
beautifulsoup4               4.12.0
blinker                      1.7.0
cachetools                   5.3.2
certifi                      2023.11.17
cffi                         1.15.1
charset-normalizer           3.1.0
click                        8.1.7
comm                         0.2.1
contourpy                    1.0.7
cryptography                 40.0.1
cycler                       0.11.0
debugpy                      1.6.7
decorator                    5.1.1
exceptiongroup               1.2.0
executing            

In [24]:
from sec_edgar_downloader import Downloader
from collections import defaultdict
from pathlib import Path
from xbrl import XBRLParser, GAAP
import pandas as pd
import os


# Define the ticker symbol and the directory to save the filings
ticker_symbol = 'AAPL'
save_dir = Path("10k_filings")

# Initialize the SEC Edgar downloader
downloader = Downloader(save_dir, 'szobev@gmail.com')

# Download the 10-K filings for the specified company
downloader.get("10-K", ticker_symbol)

# Define a function to extract financial data from the filings
def extract_financial_data(file_path):
    financial_data = defaultdict(list)
    xbrl_parser = XBRLParser()

    # Parse the XBRL data from the filing
    xbrl = xbrl_parser.parse(file_path)

    # Extract financial data from the XBRL instance
    for gaap_obj in xbrl.get_gaap():
        for concept, value in gaap_obj.items():
            financial_data[concept].append(value)

    return financial_data

# Iterate over the downloaded 10-K filings and extract financial data
for file_path in save_dir.glob(f"{ticker_symbol}/*.txt"):
    print(f"Extracting financial data from {file_path}...")
    financial_data = extract_financial_data(file_path)
    
# Print the extracted financial data
for concept, values in financial_data.items():
    print(f"{concept}: {values}")



In [18]:
# Calculate financial metrics (adapted for 10-K data)

financial_data['Revenue_Growth_Rate'] = ((financial_data['Revenue'] - financial_data['Revenue'].shift(1)) / financial_data['Revenue'].shift(1)) * 100
financial_data['Net_Profit_Margin'] = (financial_data['Net_Income'] / financial_data['Revenue']) * 100
financial_data['Debt_to_Equity_Ratio'] = financial_data['Total_Debt'] / financial_data['Total_Equity']
financial_data['Free_Cash_Flow'] = financial_data['Operating_Income'] - financial_data['Capital_Expenditures']
financial_data['ROE'] = (financial_data['Net_Income'] / financial_data['Total_Equity']) * 100
financial_data['EPS'] = (financial_data['Net_Income'] - financial_data['Preferred_Dividends']) / financial_data['Outstanding_Shares']
financial_data['Operating_Margin'] = (financial_data['Operating_Income'] / financial_data['Revenue']) * 100
financial_data['Book_Value_per_Share'] = (financial_data['Total_Equity'] - financial_data['Preferred_Dividends']) / financial_data['Outstanding_Shares']
financial_data['Dividend_Yield'] = (financial_data['Preferred_Dividends'] / financial_data['Stock_Price']) * 100
financial_data['Market_Capitalization'] = financial_data['Stock_Price'] * financial_data['Outstanding_Shares']


KeyError: 'Revenue'